In [1]:
# import statements
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import glob
import os

In [3]:


# importing a sample
arr = pd.read_csv('testWave005_mijn35_R001.csv', header=None, delimiter=";")
# check the shape
input_shape = arr.shape
print(arr.iloc[0])
print(arr.iloc[1])
print(f"The input shape of your array is: {input_shape}")

0                                     Frame
1              Elapsed_Time_In_Milliseconds
2                                      Time
3                                    Hand_X
4                                    Hand_Y
                       ...                 
362       Ring_DIP_Flex_AngularAcceleration
363    Pinky_MCP_Spread_AngularAcceleration
364      Pinky_MCP_Flex_AngularAcceleration
365      Pinky_PIP_Flex_AngularAcceleration
366      Pinky_DIP_Flex_AngularAcceleration
Name: 0, Length: 367, dtype: object
0                 0
1                 0
2      00:00:00:000
3                 0
4                 0
           ...     
362        37274,14
363       -46771,95
364        27792,87
365        87709,06
366        83074,34
Name: 1, Length: 367, dtype: object
The input shape of your array is: (283, 367)


In [18]:
folder_path = 'csv_letters/*.csv'
x = []
y = []

# Example class names
class_names = ["letterA", "letterB", "letterC","letterD","letterE","letterF","letterG","letterH","letterI","letterK","letterL","letterM","letterN","letterO","letterP","letterQ","letterR","letterS","letterT","letterU","letterV","letterW","letterX","letterY"]

# Create a dictionary mapping class names to numerical labels
class_to_label = {class_name: label for label, class_name in enumerate(class_names)}
# Use glob to get all the csv files in the folder
csv_files = glob.glob(folder_path)

# Initialize an empty list to store the combined DataFrames
combined_dataframes = []
max_len = 0
for i in range(1, len(csv_files), 2):
    # Read the first file into a DataFrame
    df1 = pd.read_csv(csv_files[i], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal='.')
    if (len(df1)>max_len):
        max_len = len(df1)
print(f"max length is {max_len}")

max length is 315


In [24]:


# Iterate over the files two by two
for i in range(1, len(csv_files), 2):
    # Read the first file into a DataFrame
    df1 = pd.read_csv(csv_files[i], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal='.')
    # Get the shape of df1
    # print(f"Shape of df1: {df1.shape}")

    #get the class and add it to y
    name = os.path.splitext(csv_files[i])
    classtype = name[0].split('\\')[1][:7] #this takes letterX from the title
    numerical_label = class_to_label.get(classtype, -1)  # -1 if not found
    if numerical_label != -1:
        y.append(numerical_label)
    else:
        print("failed to add to y: " + classtype)

    # If there is a next file, read it and combine with the first
    if i+1 < len(csv_files):
        df2 = pd.read_csv(csv_files[i+1], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal='.')
        df2 = df2.add_prefix('right_')
    else:
        # If there is no next file, create an empty DataFrame with the same columns as df1
        df2 = pd.DataFrame(columns=df1.columns)

    # Ensure that both DataFrames have the same number of rows by filling NaN values in df2
    if len(df1) > len(df2):
        df2 = df2.reindex(df1.index, fill_value=np.nan)
    elif len(df2) > len(df1):
        df1 = df1.reindex(df2.index, fill_value=np.nan)

    # Concatenate the DataFrames
    combined_df = pd.concat([df1, df2], axis=1)
    combined_df = combined_df.reindex(range(max_len)).fillna(0)
    # Print the shape of combined_df
    print(f"Shape of combined_df: {combined_df.shape}")
    combined_dataframes.append(combined_df)

# Convert the list of DataFrames to a NumPy array
x = np.array(combined_dataframes)
y = np.array(y)
print(x)
print(y)



Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of c

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1530,) + inhomogeneous part.

In [23]:
model = models.Sequential()
model.add(layers.Conv1D(32, 3, activation='relu', input_shape=(282,366)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 3, activation='relu'))

In [24]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(26))

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 280, 32)           35168     
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 140, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_7 (Conv1D)           (None, 138, 64)           6208      
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 69, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_8 (Conv1D)           (None, 67, 64)            12352     
                                                                 
 flatten_3 (Flatten)         (None, 4288)             

In [26]:
model.compile(optimizer='adam',  # we used adam during machine learning course
              loss='categorical_crossentropy',  # Multiclass classification loss
              metrics=['accuracy'])  # Track accuracy during training
print(x.size)
print(y.size)
model.fit(x,y,epochs=50)

0
0


ValueError: Expected input data to be non-empty.